In [ ]:
## PURPOSE: 

In [ ]:
## TODO

(1)
salience1, salience2, salience3 -> eu_salience DONE
dissent1, dissent2, dissent3 -> eu_dissent DONE
require_econ,require_pol,require_gov ->  DONE (check on email, not 100% clear that 2019 version *is* 0-10 not 1-7)
Requires transforming scales to match

(2) Policy variables matched DONE

(3) Harmonise std_X X_sd DONE

(4) Country codes/cname/country_abbrev -> Harmonise DONE!

(5) Look at pattern of NULLS -> Maybe check against expert-level datasets?
    # grabbed the datasets ... prob not going to be quick

(6) Look at changes made in the 1999-2014 trend file - if they look like fixes, try to incorporate
# few issues with changed party_ids -> DONE
# some extra columns they filled in from parlgov and the cmp_ids! -> add NEXT

(7) normalise everything to [0-1] and run dtype/range testing

(8) Add all notes to email to CHES

(9) tidy previous datasets notebook, tidy this notebook, share with CHES

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re, os, pickle

In [2]:
dataset_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\datasets\\"
output_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\output\\"

In [3]:
def could_be_int(x):
    if isinstance(x,float) or isinstance(x,int):
        return True
    else:
        try:
            int(x.strip())
            return True
        except:
            return False
        
def could_not_be_int(x):
    if isinstance(x,float) or isinstance(x,int):
        return False
    else:
        try:
            int(x.strip())
            return False
        except:
            return True        
        
def describe_messy_column(ser):

    nulls = ser.isnull().sum()
    print(nulls," nulls")
    ser = ser[ser.notnull()]
    vc = ser[ ser.apply(lambda x: could_not_be_int(x))].value_counts()
    display(vc[vc>0])
    print("min: ",ser[ ser.apply(lambda x: could_be_int(x))].min(),"max: ",ser[ ser.apply(lambda x: could_be_int(x))].max())
        

In [4]:
CHES_replace_dict = {
 # CHES_2007_CAND 
 # membership
 'neutral':4.0,
 # salience2
 'little importance':2.0,
 'some importance':3.0,
 'great importance':4.0,
 # require_pol
#  'neutral':4.0,
 # require_gov
#  'neutral':4.0,
 # minor
 'strongly supports more rights for ethnic minorities':1.0,
 
    
    
 # CHES_1999_2014
 # eu_ep
 'strongly opposes expanding EP powers':1.0,
 'neutral towards expanding EP powers':4.0,
 'strongly favors expanding EP powers':7.0,
 # eu_fiscal
 'strongly opposes tax harmonization':1.0,
 'neutral towards tax harmonization':4.0,
 'strongly favors tax harmonization':7.0,
 # eu_intmark
 'stongly opposes expanding EU powers on the internal market':1.0,
 'neutral toward expanding EU powers on the internal market':4.0,
 'stongly favors explanding EU powers on the internal market':7.0,
 # eu_employ
 'strongly opposes a common employment policy':1.0,
 'neutral towards a common employment policy':4.0,
 # eu_agri
 'strongly opposes agricultural policy':1.0,
 'neutral toward agricultural policy':4.0,
 'strongly favors agricultural policy':7.0,
 # eu_cohesion
 "strongly opposes the EU's cohesion policy":1.0,
 "neutral towards the EU's cohesion policy":4.0,
 "strongly favors the EU's cohesion policy":7.0,
 # eu_environ
 'strongly opposes a common policy on environment':1.0,
 'neutral towards a common policy on environment':4.0,
 'strongly favors a common policy on environment':7.0,
 # eu_asylum
 'strongly opposes a common policy on political asylum':1.0,
 'neutral towards a common policy on political asylum':4.0,
 'strongly favors a common policy on political asylum':7.0,
 # eu_foreign
 'strongly opposes a common policy on political asylum':1.0,
 'neutral towards a common policy on political asylum':4.0,
 'strongly favors a common policy on political asylum':7.0,
 # eu_turkey
 'strongly opposes':1.0,
#  'neutral':4.0,
 # cosmo
 # position
 'strongly opposed':1.0,
 'strongly in favor':7.0,
    
    
    
    
 # CHES_1984_1999
 # position
'strongly opposed to ei':1.0,
'neutral, no stance on ei':4.0,
'strongly in favor of ei':7.0,
 # foreign
 'strongly opposes a common foreign and security policy':1.0,
 'neutral towards a common foreign and security policy':4.0,
 'strongly favors a common foreign and security policy':7.0,
 # salience1
 'no importance, never mentioned':1.0,
 'minor issue':2.0,
 'important issue':3.0,
 'one of the most important issues':4.0,
 'the most important issue for the party':5.0,
 # dissent2
 'complete unity':1.0,
 'minor dissent':2.0,
 'significant dissent':3.0,
 'evenly split':4.0,
 # future
 'unification has gone much too far':1.0,
 'just the right amount of unification':3.0,
 # ep
 'strongly opposed expanding ep powers':1.0,
 'neutral towards expanding ep powers':4.0,
 'strongly favors expanding ep powers':7.0,
 # fiscal
 'strongly opposes tax harmonization':1.0,
 'neutral towards tax harmonization':4.0,
 'strongly favors tax harmonization':7.0,
 # employ
 'strongly opposes a common employment policy':1.0,
 'neutral towards a common employment policy':4.0,
 # cohesion
 "strongly opposes the eu's cohesion policy":1.0,
 "neutral towards the eu's cohesion policy":4.0,
 "strongly favors the eu's cohesion policy":7.0,
 # environ
 'strongly opposes a common policy on environment':1.0,
 'neutral towards a common policy on environment':4.0,
 'strongly favors a common policy on environment':7.0,
 # asylum
 'strongly opposes a common policy on political asylum':1.0,
 # lrgen
 'center':5.0,
 # lrecon
#  'center':5.0,
 # galtan
#  'center':5.0,
 'extreme tan':10.0,
 # lr_gen
 'extreme left':0.0,
 'extreme right':1.0,
    
    ## PROBLEM - neutral here is on a different scale than all other neutrals!
 # pro_anti
#  'anti european integration':1.0,
#  'neutral':2.0,
#  'pro european integration:':3.0,

}

In [5]:
# amalgamate all means!

# then look at correlation to see if any columns *really* ought to be merged

CHES_1984_1999 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "1984-1999_dataset_means.dta")
CHES_1984_1999["source"] = "1984-1999_dataset_means.dta"
CHES_1999_2014 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "1999-2014_CHES_dataset_means-3.dta")
CHES_1999_2014["source"] = "1999-2014_CHES_dataset_means-3.dta"
CHES_2007_CAND = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2007_CHES-candidates_dataset_means.dta")
CHES_2007_CAND["source"] = "2007_CHES-candidates_dataset_means.dta"
CHES_2014_UKR = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2014_Candidates_Ukraine_Means.dta")
CHES_2014_UKR["source"] = "2014_Candidates_Ukraine_Means.dta"
CHES_2017 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "CHES_means_2017.dta")
CHES_2017["source"] = "CHES_means_2017.dta"
CHES_2019 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "CHES2019V3.dta")
CHES_2019["source"] = "CHES2019V3.dta"

pro_anti_replace_dict = { 'anti european integration':1.0,
 'neutral':2.0,
 'pro european integration':3.0,}

# 'neutral' appears on a different scale to all other mentions
CHES_1984_1999["pro_anti"] = CHES_1984_1999["pro_anti"].replace(pro_anti_replace_dict)

# add year values to CHES_2019 and CHES_2014_UKR (questionnaires specify 2014/2019)
CHES_2019["year"] = 2019
CHES_2014_UKR["year"] = 2014

# seems to confusion over whether party_id 1210 is PDC (party of christian democracy) or PCP (portuges communist party)
CHES_2019.loc[103,"party_id"] = 1209


CHES_comb = pd.concat([CHES_1984_1999,CHES_1999_2014,CHES_2007_CAND,CHES_2014_UKR,CHES_2017,CHES_2019],axis=0)
CHES_comb = CHES_comb.replace(CHES_replace_dict)

CHES_comb = CHES_comb.reset_index()
CHES_comb = CHES_comb.rename(columns = {"index":"source_index"})

# complete
# party_id               2145
# country                2145


# chesversion     949
# lrecon          949
# eu_foreign      949
# govt            949
# family          949
# electionyear    949
# party           949
# position        949
# expert          949
# year            949
# eumember        949
# eastwest        949
# country         949

# galtan          948
# eu_salience     945
# eu_dissent      945
# eu_ep           942
# eu_cohesion     941
# vote            899
# seat            895
# eu_intmark      807
# epvote          773

# cmp_id          797

# eu_econ_require, eu_political_require, eu_googov_require -> only 5 nonnull entries each!

# multiculture_dissent       0
# multiculture_salience      0
# multiculture               0
# eu_budget                  0

In [6]:
eumember_replace_dict = {"EU member state":"Yes",
                       1:"Yes",
                       0:"No",
                       "other":"Other" }

CHES_comb["eumember"] = CHES_comb["eumember"].replace(eumember_replace_dict)


eastwest_replace_dict = { "western europe":1.0,
                          "west":1.0,
                          "east":0.0,
}

CHES_comb["eastwest"] = CHES_comb["eastwest"].replace(eastwest_replace_dict)


govt_replace_dict = { "not in government":0.0,
                          "in government":1.0,
}

CHES_comb["govt"] = CHES_comb["govt"].replace(govt_replace_dict)


family_replace_dict = {"1. radical TAN":"rad right",
 "2. conservative":"cons",
 "3. liberal":"liberal",
 "4. Christian-democratic":"christdem",
 "5. socialist":"socialist",
 "6. radical left":"rad left",
 "7. green":"green",
 "8. regionalist":"regionalist",
 "9. no family":"no family",
 "10. confessional":"confessional",
 "11. agrarian/center":"agrarian",
 "agrarian/centre":"agrarian",
 "rad right ":"rad right",
 "protestant":"confessional", # not 100% certain about this! theoretically, you can have confessional parties that aren't prot

}
CHES_comb["family"] = CHES_comb["family"].replace(family_replace_dict)


### all to get nicely consistent dummies most important issue variables
df = pd.DataFrame()

mip_replace_dict = {"public services vs taxes":"services vs taxes",
                    "anti-elite rhetoric":"anti-elite",
                    "ethnic minorities":"ethnic", # change these back to long-form later
                    "EU integration":"EU",# change these back to long-form later
                   }

base_list = list(CHES_comb[["mip_one","mip_two","mip_three"]].replace(mip_replace_dict).stack().value_counts().index[0:18])

base_list = pd.Series(base_list).replace(mip_replace_dict).values

CHES_comb[["mip_one","mip_two","mip_three"]] = CHES_comb[["mip_one","mip_two","mip_three"]].replace(mip_replace_dict)

def mip_transform(x):
    if pd.isnull(x):
        return np.nan
    elif "tie:" in x:
        return x.replace("tie: ","").split(" and ")
    else:
        return [x]

long_form_dict ={"EU":"EU integration","ethnic":"ethnic minorities"}
    
for cat in base_list:
    for mip_pos in ["one","two","three"]:
        if cat in long_form_dict.keys():
            long_cat = long_form_dict[cat]
        else:
            long_cat = cat
        df["mip_"+mip_pos+"__"+long_cat] = CHES_comb["mip_"+mip_pos]\
                                    .apply(lambda x: mip_transform(x))\
                                    .apply(lambda x: np.nan if x is np.nan else cat in x)\
                                    .astype('float32')

CHES_comb[df.columns] = df        
CHES_comb = CHES_comb.drop(["mip_one","mip_two","mip_three"],axis=1)
#####

df = pd.get_dummies(CHES_comb["family"],prefix="family", prefix_sep='__')
df.loc[CHES_comb["family"].isnull(),:] = np.nan
CHES_comb[df.columns] = df        
df = pd.get_dummies(CHES_comb["eumember"],prefix="eumember", prefix_sep='__')
df.loc[CHES_comb["eumember"].isnull(),:] = np.nan
CHES_comb[df.columns] = df        
    
CHES_2014_UKR_country_replace_dict = {30.0:"bih",32.0:"mkd",33.0:'ser',41.0:'alb',42.0:'kos',43.0:'mne',44.0:'ukr'}
CHES_2007_CAND_country_replace_dict = {'      30':'bih', '      31':'cro', '      32':'mkd', '      33':'ser', '      34':'tur'}    
CHES_comb["country"] = CHES_comb["country"].replace(CHES_2014_UKR_country_replace_dict)
CHES_comb["country"] = CHES_comb["country"].replace(CHES_2007_CAND_country_replace_dict)


CHES_comb["party_name"] = CHES_comb["party_name"].replace("",np.nan)
CHES_comb = CHES_comb.drop( ["cname","country_abb"] , axis=1 )

# no entries, all from CHES_2017
CHES_comb = CHES_comb.drop( ["multiculture_dissent","multiculture_salience","multiculture","eu_budget",] , axis=1 )

# var00001 appears to be just an early version of party_id
CHES_comb = CHES_comb.drop( ["var00001",] , axis=1 )


# 1984-1999_dataset_means - dummied values for country, seems a bit premature/unnecessary to me given we have a complete column for country
CHES_comb = CHES_comb.drop( ['aus', 'be', 'dk', 'esp', 'fin', 'fr', 'ger', 'gr', 'irl', 'it', 'nl', 'por', 'sw', 'uk',]
                           , axis=1 )

# confusingly, we have both a family column *and dummies* - all in this dataset
CHES_comb = CHES_comb.drop( ['radrt', 'con', 'lib', 'cd', 'soc', 'radleft', 'green', 'reg',] , axis=1 )

# party_name is just "party" for 2014_Candidates_Ukraine_Means
CHES_comb.loc[CHES_comb["party_name"].notnull(),"party"] = CHES_comb.loc[CHES_comb["party_name"].notnull(),"party_name"]
CHES_comb = CHES_comb.drop( ["party_name"] , axis=1 )

# chesversion only present in 1999-2014_CHES_dataset_means-3.dta (1.1), changing to string to avoid any confusion
# with numerical data
CHES_comb["chesversion"] = CHES_comb["chesversion"].apply(lambda x: str(round(x,1)) if pd.notnull(x) else np.nan)

# experts, eastwest, eumember (and eumember__(Yes/No/Other), 
# party_id,cmp_id
    # not one-to-one, mostly one-to-one, some many-to-one and
    # *one case* where cmp has one reference for a Croatian Party of Rights and it's splinter, but CHES has two
# country, party, partyname1, partyname2, year, family (and family__(dummied options),
# electionyear, vote, seat, epvote,
# source, source_index, chesversion

# everything else is a (meaned) 'expert' evaluation

# looking at correlations to find redundant variable
# we get 100% correlation between lrgen and lr_gen in the 141 instances where they overlap
# going to go ahead and amalgamate them and drop lr_gen
CHES_comb.loc[CHES_comb["lrgen"].isnull(),"lrgen"] = CHES_comb.loc[CHES_comb["lrgen"].isnull(),"lr_gen"]*10
CHES_comb = CHES_comb.drop("lr_gen",axis=1)

# save it as a pickle!
CHES_comb.to_pickle(dataset_folder+"CHES"+"CHES_comb.zip",compression='zip')

In [399]:
corrs = CHES_comb.corr()
corrs = corrs.where(np.triu(np.ones(corrs.shape),k=2).astype(np.bool))
df = corrs.stack().reset_index()
df.columns = ['Row','Column','Value']
# df.drop(df.index[df["Value"]==1.0],inplace=True)

df["N"] = df.apply(lambda x: (CHES_comb[x["Row"]].notnull()&CHES_comb[x["Column"]].notnull()).sum(),axis=1)

df.sort_values(by="Value")

,Row,Column,Value,N
4401,civlib_laworder,eu_political_require,-0.996822,5
1676,galtan,eu_political_require,-0.994686,5
6330,conflict_res,russia_role,-0.987668,12
6329,conflict_res,languages,-0.987145,12
4218,redistribution,eu_political_require,-0.984077,5
...,...,...,...,...
5385,enviro_salience,eu_googov_require,0.979922,5
2600,eu_dissent,eu_econ_require,0.985288,5
6139,corrupt_salience,eu_googov_require,0.989326,5
4332,redist_salience,eu_political_require,0.992014,5


In [406]:
CHES_comb[["party_id"]+[x for x in CHES_comb.columns if "blur" in x or "clear" in x]].dropna(how='all').groupby("party_id").mean().corr()

,lrecon_clear,galtan_clear,galtan_new_clear,eu_blur,lrecon_blur,galtan_blur
lrecon_clear,1.000000,0.276668,0.076753,-0.141007,-0.668113,-0.037772
galtan_clear,0.276668,1.000000,0.817016,-0.080838,-0.095085,-0.693077
galtan_new_clear,0.076753,0.817016,1.000000,-0.072746,0.090751,-0.610898
eu_blur,-0.141007,-0.080838,-0.072746,1.000000,0.338106,0.197513
lrecon_blur,-0.668113,-0.095085,0.090751,0.338106,1.000000,0.319051
galtan_blur,-0.037772,-0.693077,-0.610898,0.197513,0.319051,1.000000


In [798]:
# # 'eu_econ_req', 'eu_pol_req','eu_good_gov': 1-7 (2010 as 1-7, 2019 as 0-10)
# # 'require_econ', 'require_pol', 'require_gov': 0-10 (CHES_2007_CAND 1-7,CHES_2014_UKR -17)
# CHES_2019 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "CHES2019V3.dta")
# CHES_2019["source"] = "CHES2019V3.dta"

# CHES_2007_CAND = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2007_CHES-candidates_dataset_means.dta")
# CHES_2007_CAND["source"] = "2007_CHES-candidates_dataset_means.dta"

# CHES_2014_UKR = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2014_Candidates_Ukraine_Means.dta")
# CHES_2014_UKR["source"] = "2014_Candidates_Ukraine_Means.dta"
# # 
# [x for x in CHES_2007_CAND.columns if "req" in x]
# # 2019 ['eu_econ_require', 'eu_political_require', 'eu_googov_require']
# # 2010 ['eu_econ_req', 'eu_pol_req', 'eu_good_gov']
# # 2014_UKR ['require_econ', 'require_pol', 'require_gov']
# # 2007_CAND ['require_econ', 'require_pol', 'require_gov']

[x for x in CHES_2019.columns if "req" in x]

['eu_econ_require', 'eu_political_require', 'eu_googov_require']

In [775]:
CHES_1999 = pd.read_stata(dataset_folder+"CHES"+os.sep+"trend files"+os.sep+"1999_CHES_dataset_means.dta")
CHES_1999["source"] = "1999_CHES_dataset_means.dta"

CHES_2002 = pd.read_stata(dataset_folder+"CHES"+os.sep+"trend files"+os.sep+"2002_CHES_dataset_means.dta")
CHES_2002["source"] = "2002_CHES_dataset_means.dta"

CHES_2006 = pd.read_stata(dataset_folder+"CHES"+os.sep+"trend files"+os.sep+"2006_CHES_dataset_means.dta")
CHES_2006["source"] = "2006_CHES_dataset_means.dta"

CHES_2010 = pd.read_stata(dataset_folder+"CHES"+os.sep+"trend files"+os.sep+"2010_CHES_dataset_means.dta")
CHES_2010["source"] = "2010_CHES_dataset_means.dta"

CHES_2014 = pd.read_stata(dataset_folder+"CHES"+os.sep+"trend files"+os.sep+"2014_CHES_dataset_means-2.dta")
CHES_2014["source"] = "2014_CHES_dataset_means-2.dta"

In [776]:
redundant_dummies = ['gr', 'wall', 'lith', 'lat', 'sw', 'dk', 'rom', 'lib', 'por', 'est', 'be', 'nl', 'hung', 'esp',
 'reg', 'bul', 'confess', 'radrt', 'green', 'fl', 'uk', 'ger', 'irl', 'fin', 'it', 'aus', 'pol', 'bel', 'czech',
 'nofamily', 'fr', 'slovak', 'con', 'radleft', 'sv', 'sloven', 'soc', 'agrarian', 'cd', # unnecessary dummy columns
                    'pro_anti','lr_gen','var00001', # derived/replicated columns
                    'multiculture', 'multiculture_salience', 'multiculture_dissent',# 2017 flash dataset bugged empty columns
                    ]

column_name_replace ={"asylum":"eu_asylum",
                      "cohesion":"eu_cohesion",
                      "dissent":"eu_dissent",
                      "employ":"eu_employ",
                      "environ":"eu_environ",
                      "ep":"eu_ep",
                      "fiscal":"eu_fiscal",
                      "foreign":"eu_foreign",
                      "salience":"eu_salience",
                     
                      "agri":"eu_agri",
                      "part_id":"party_id",
                      
                      'pubserv_sal': 'pubserv_salience',
                      'dereg_sal': 'dereg_salience',
                      'redist_sal': 'redist_salience',
                      'civlib_sal': 'civlib_salience',
                      'lifestyle_sal': 'lifestyle_salience',
                      'religion_sal': 'religion_salience',
                      'immig_sal': 'immig_salience',
                      'multicult_sal': 'multicult_salience',
                      'urban_sal': 'urban_salience',
                      'cosmo_sal': 'cosmo_salience',
                      'decentral_sal': 'decentral_salience',
                      'us_sal': 'us_salience',
                      'minor_sal': 'minor_salience',                      
                      
                      "turkey":"eu_turkey",
                      "intmark":"eu_intmark",
                      
                      "position":"eu_position",
                      
                      'immasylum_salience':'multicult_salience',
                      'spendvtax':'pubserv', 'deregulation':'dereg', 'redistribution':'redist',
                      'civlib_laworder':'civlib', 'sociallifestyle':'lifestyle',
                      'religious_principle':'religion', 'immigrate_policy':'immig', 'multiculturalism':'multicult',
                      'urban_rural':'urban',  'regions':'decentral', 'ethnic_minorities':'minor',
                      'nationalism':"cosmo",
                      
                      "spendvtax_salience":"pubserv_salience",
                      "social_salience":"lifestyle_salience",
                      "relig_salience":"religion_salience",
                      "immigra_salience":"immig_salience",
                      "region_salience":"decentral_salience",
                      "ethnic_salience":"minor_salience",
                      
                      'immigrate_salience':'immig_salience','immigrate_dissent':'immig_dissent','environment':'enviro',
#                       'eu_econ_require':'eu_econ_req', 'eu_political_require':'eu_pol_req', 'eu_googov_require':'eu_good_gov',
                      'require_econ':"eu_econ_req",'require_pol':'eu_pol_req','require_gov':'eu_good_gov',
                      'religious_principles':'religion',

                      'dissep': 'eu_ep_dissent',
                      'dissintm': 'eu_intmark_dissent',
                      'dissempl': 'eu_employ_dissent',
                      'dissagri': 'eu_agri_dissent',
                      'disscohe': 'eu_cohesion_dissent',
                      'dissenv': 'eu_environ_dissent',
                      'dissasyl': 'eu_asylum_dissent',
                      'dissenlw': 'enlargw_dissent',
                      'dissenle': 'enlarge_dissent',

                      'experts':'expert',# 2017 flash dataset
                      
                      'std_position': 'position_sd',
                      'std_lrgen': 'lrgen_sd',
                      'std_lrecon': 'lrecon_sd',
                      'std_galtan': 'galtan_sd',
                      'std_salience2': 'salience2_sd',
                      'std_dissent3': 'dissent3_sd',
                      'std_membership': 'membership_sd'                      
                     }



set.intersection(set(CHES_1999), set(CHES_2002), set(CHES_2006),set(CHES_2010),set(CHES_2014))

{'country', 'galtan', 'lrecon', 'lrgen', 'source'}

In [777]:
CHES_1999.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_1999 = CHES_1999.rename(columns = column_name_replace)
CHES_1999.shape

# common to all 1999-2014 datasets - 'country', 'galtan', 'lrecon', 'lrgen', 'source', (party_id)

# eastwest: 'west' (all!)
# eumember: 'eu member state' (all!)
# country: ['be', 'dk', 'ge', 'gr', 'esp', 'fr', 'irl', 'it', 'nl', 'uk', 'por', 'aus', 'fin', 'sv']
# year: 1999 (all!)
# source: 1999_CHES_dataset_means.dta
# expert: 5-15
# party_id: unique int id
CHES_1999["party_id"] = CHES_1999["party_id"].astype(int)
# FIXED: on the basis of changes made in the 1999_2014 trend file!
CHES_1999.loc[CHES_1999["party_id"]==710,"party_id"] = 708
# party: non-unique party abbreviation
# vote: voteshare in last national election (estimated via seat proprtion where not available for coalition partners)
    # quite a lot of 0%s
# family:['rad right ', 'cons', 'liberal', 'christdem', 'socialist', 'rad left',
#        'green', 'regionalist', 'no family', 'protestant', 'agrarian'],
# govt: categories mislabelled - read ['in government part of year', 0.5, 'in government']
    # should read ['Party not in government'(/0) ,in government part of year'(/0.5), 'in government'(/1.0)]
    # specific example party 1101 (UK Conservative) is listed as 'in government part of year'
    # entries with .5 check out (verhofstadt's Belgium 1999 6 party coalition!), as does 'in government' entries
    # note - 1999 codebook contains no 'govt' reference!
    # FIX {'in government part of year':0.0, 0.5:0.5, 'in government':1.0}
    
CHES_1999["govt"] = CHES_1999["govt"].replace( {'in government part of year':0.0, 0.5:0.5, 'in government':1.0} )

CHES_1999.shape,CHES_1999.columns

['eastwest', 'eumember', 'country', 'year', 'expert', 'party_id',
       'party', 'vote', 'family', 'govt', 'position', 'salience1', 'dissent1',
       'future', 'eu_ep', 'eu_fiscal', 'eu_employ', 'eu_cohesion',
       'eu_environ', 'eu_asylum', 'eu_foreign', 'lrgen', 'lrecon', 'galtan',
       'pro_anti', 'source'],

var1 = "galtan"
CHES_1999[var1][ CHES_1999[var1].apply(lambda x: could_not_be_int(x))].value_counts(),CHES_1999[var1][ CHES_1999[var1].apply(lambda x: could_be_int(x))].min(),CHES_1999[var1][ CHES_1999[var1].apply(lambda x: could_be_int(x))].max()

#### expert responses meaned

# position
# 1-7
# {"strongly in favor":1,
#  "neutral":4,
#  "strongly opposed":7,}

# salience1
# 1-5
# {"one of the most important issues":4,"important issue":3,"minor issue":2}

# dissent1
# 1-5
# {"complete unity":1,"minor dissent":2,'significant dissent':3}

# future
# 1-5
# {"unification has gone much too far":1, "just the right amount of unification":3}

# eu_ep
# 1-7 (8 is code for 'missing')
# {"neutral towards expanding ep powers":4,"strongly favors expanding ep powers":7, "strongly opposes expanding ep powers":1}

# eu_fiscal
# 1-7 (8 is code for 'missing')
# {"strongly opposes tax harmonization":1,"neutral towards tax harmonization":4,"strongly favors tax harmonization":7}

# eu_employ
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common employment policy":1,"neutral towards a common employment policy":4}

# eu_cohesion
# 1-7 (8 is code for 'missing') - codebook has a repeated label typo here!
# {"strongly opposes the eu's cohesion policy":1,"neutral towards the eu's cohesion policy":4,"strongly favors the eu's cohesion policy":7}

# eu_environ
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common policy on environment":1,"neutral towards a common policy on environment":4,"strongly favors a common policy on environment":7}

# eu_asylum
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common policy on political asylum":1}

# eu_foreign
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common foreign and security policy":1,"neutral towards a common foreign and security policy":4,"strongly favors a common foreign and security policy":7}

# lrgen
# 0-10
# {"center":5}

# lrecon
# 0-10
# {"center":5}

# galtan
# 0-10
# {"center":5}

# pro_anti
# drop - it's just position coded to 3 categories!

# no missing values (suspicious!) - not even party == ""


(center                3
 5.125                 0
 3.375                 0
 3.4000000953674316    0
 3.4285714626312256    0
                      ..
 6.7272725105285645    0
 6.800000190734863     0
 6.833333492279053     0
 7.0                   0
 0.6000000238418579    0
 Name: galtan, Length: 97, dtype: int64,
 0.6000000238418579,
 9.75)

In [778]:

CHES_2002.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2002 = CHES_2002.rename(columns = column_name_replace)
CHES_2002.shape,CHES_2002.columns
['eastwest', 'eumember', 'country', 'expert', 'year', 'party_id',
        'party', 'partyname1', 'partyname2', 'vote', 'vote1', 'family', 'govt',
        'position', 'std_position', 'salience2', 'eu_ep', 'eu_intmark',
        'eu_employ', 'eu_agri', 'eu_cohesion', 'eu_environ', 'eu_asylum',
        'eu_foreign', 'enlargw', 'enlarge', 'dissent2', 'dissep', 'dissintm',
        'dissempl', 'dissagri', 'disscohe', 'dissenv', 'dissasyl', 'dissenlw',
        'dissenle', 'lrgen', 'lrecon', 'galtan', 'std_lrgen', 'std_lrecon',
        'std_galtan', 'source'],

# NULLS: only in dissenlw        73, dissenle        98
# enlargw, enlarge (but they were coded as 99!)

# eastwest: ['west','east',]
# eumember: 0|1
# country: ['be', 'dk', 'ge', 'gr', 'esp', 'fr', 'irl', 'it', 'nl', 'uk', 'por',
#        'aus', 'fin', 'sv', 'bul', 'czech', 'hung', 'lat', 'lith', 'pol', 'rom',
#        'slovak', 'sloven']
# year: 2002 (all!)
# source: 2002_CHES_dataset_means.dta
# expert: 4-18
# party_id: unique int id
# party: non-unique party abbreviation
# partyname1, partyname2: longer string versions (original/English)
# vote: voteshare in last national election (estimated via seat proprtion where not available for coalition partners)
    # no 0%s here
# vote1: same, but for the overall coalition if party is in one (why not 'coalitional_vote'???)

# so vote1 >= vote - which is it, except in one case!!!
# CHES_2002[(CHES_2002["vote"]-CHES_2002["vote1"])>0][CHES_2002.columns[0:20]]
# 2701, https://en.wikipedia.org/wiki/2000_Romanian_general_election
# FIX! vote1 looks about right for the coalition, but obvously vote should be smaller
# Alliance was pretty asymmetric, can't see how seats were divided up, but I wouldn't imagine vote is much smaller than vote1


# family:['radical right', 'cons', 'liberal', 'christdem', 'socialist',
#        'radical left', 'green', 'region/ethnic', 'no family', 'confessional',       'agrarian']
#     FIX - categories need to be renamed!
# govt: categories mislabelled - read ['in government part of year', 0.5, 'in government']
    # SAME PROBLEM, different strings
    # should read ['Party not in government'(/0) ,in government part of year'(/0.5), 'in government'(/1.0)]
    # specific example party 1101 (UK Conservative) is listed as 'in government part of year'
    # entries with .5 check out (verhofstadt's Belgium 1999 6 party coalition!), as does 'in government' entries
    # note - 1999 codebook contains no 'govt' reference!
    # FIX {'in government part of year':0.0, 0.5:0.5, 'in government':1.0}

CHES_2002["govt"] = CHES_2002["govt"].replace( {'party in government for part of the time point of evaluation':0.0, 0.5:0.5, 'party in government during the year of evaluation':1.0} )


#### expert responses meaned

# position
# 1-7
# {"strongly in favor":1, "neutral":4, "strongly opposed":7,
#   "strongly opposed to european integration":1,"neutral towards european integration":4,"strongly in favor of european integration":7}
# - std_position

# salience2 (not salience 1 - different range)
# 1-4
# {"one of the most important issues":4,"important issue":3,"minor issue":2} salience1
# {"little importance":2,"some importance":3,"great importance":4} salience2

# dissent2 (not dissent1(1-5))
# 1-10
# {"complete unity":1,"minor dissent":2,'significant dissent':3} dissent1
# no strings in dissent2


# future DISCONTINUED
# 1-5
# {"unification has gone much too far":1, "just the right amount of unification":3}

# eu_ep
# 1-7 (8 is code for 'missing')
# {"strongly opposes expanding ep powers":1,"neutral towards expanding ep powers":4,"strongly favors expanding ep powers":7}
# {"strongly opposed expanding ep powers":1,"neutral towards expanding ep powers":4}

# eu_fiscal NOT PRESENT
# 1-7 (8 is code for 'missing')
# {"strongly opposes tax harmonization":1,"neutral towards tax harmonization":4,"strongly favors tax harmonization":7}

# eu_employ
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common employment policy":1,"neutral towards a common employment policy":4}

# eu_cohesion
# 1-7 (8 is code for 'missing') - codebook has a repeated label typo here!
# {"strongly opposes the eu's cohesion policy":1,"neutral towards the eu's cohesion policy":4,"strongly favors the eu's cohesion policy":7}

# eu_environ
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common policy on environment":1,"neutral towards a common policy on environment":4,"strongly favors a common policy on environment":7}

# eu_asylum
# 1-7 (8 is code for 'missing')
# expanded list
# {"strongly opposes a common policy on political asylum":1,"neutral towards a common policy on political asylum":4,"strongly favors a common policy on political asylum":7}

# eu_foreign
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common foreign and security policy":1,"neutral towards a common foreign and security policy":4,"strongly favors a common foreign and security policy":7}

# lrgen
# 0-10
# {"center":5}
# - std_lrgen

# lrecon
# 0-10
# {"center":5}
# - std_lrecon

# galtan
# 0-10
# {"center":5}
# - std_galtan

# new!
# eu_intmark
# 1-7 (8 is code for missing)
# {"neutral towards internal market":4}

# eu_agri
# 1-7 (8 is code for missing)
# {"strongly opposes agricultural policy":1,"neutral toward agricultural policy":4,"strongly favors agricultural policy":7}

# enlargw (For EU-15 parties: position of the party leadership in 2002 on enlargement)
# 1-7 (8 is code for missing)
# {"neutral":4}
# FIX appears to include some "99.0" values
CHES_2002["enlargw"] = CHES_2002["enlargw"].replace(99,np.nan).replace({"neutral":4})

# enlarge (For CEE-applicants position of the party leadership in 2002 on major domestic reforms to enter EU)
# 1-7 (8 is code for missing)
# {"neutral towards eu membership":4, "strongly supports eu membership":7}

CHES_2002["enlarge"] = CHES_2002["enlarge"].replace(99,np.nan).replace({"neutral towards eu membership":4, "strongly supports eu membership":7})

## UNIQUE TO THIS DATASET
# I feel like these would be more readable as diss_ep, diss_intm ...
diss_vars = [x for x in CHES_2002.columns if "diss_" in x]
# diss_var_rename_dict = {k:"diss_"+k.split('diss')[1] for k in diss_vars}
# CHES_2002.rename(columns=diss_var_rename_dict,inplace=True)
        
# diss variables: "has this issue caused divisions within the leadership in 2002?"
# *number of experts checking box* (no constraint on how many boxes they can check)

CHES_2002[diss_vars] = CHES_2002[diss_vars].apply(lambda x: x/CHES_2002["expert"])
# normalised
        

# var1 = "diss_ep"
# CHES_2002[var1][ CHES_2002[var1].apply(lambda x: could_not_be_int(x))].value_counts(),CHES_2002[var1][ CHES_2002[var1].apply(lambda x: could_be_int(x))].min(),CHES_2002[var1][ CHES_2002[var1].apply(lambda x: could_be_int(x))].max()


In [779]:

CHES_2006.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2006 = CHES_2006.rename(columns = column_name_replace)
CHES_2006.shape,CHES_2006.columns

['eastwest', 'eumember', 'country', 'country_abb', 'year', 'expert',
        'party_id', 'party', 'partyname1', 'partyname2', 'vote', 'vote1',
        'family', 'intgroup', 'epgroup', 'govt', 'govt1', 'govt2', 'govt3',
        'position', 'std_position', 'salience2', 'std_salience2', 'dissent3',
        'std_dissent3', 'benefit', 'eu_ep', 'eu_intmark', 'eu_cohesion',
        'eu_foreign', 'eu_turkey', 'lrgen', 'std_lrgen', 'lrecon', 'std_lrecon',
        'galtan', 'std_galtan', 'var00001', 'pubserv', 'pubserv_salience', 'dereg',
        'dereg_salience', 'redist', 'redist_salience', 'civlib', 'civlib_salience',
        'lifestyle', 'lifestyle_salience', 'religion', 'religion_salience', 'immig',
        'immig_salience', 'multicult', 'multicult_salience', 'urban', 'urban_salience',
        'cosmo', 'cosmo_salience', 'decentral', 'decentral_salience', 'us', 'us_salience',
        'minor', 'minor_salience', 'source'],



# eastwest: ['west','east',]
# eumember: ['eu member state','other'] (other= bulgaria and romania)
# country: ['be', 'dk', 'ge', 'gr', 'esp', 'fr', 'irl', 'it', 'nl', 'uk', 'por',
#        'aus', 'fin', 'sv', 'bul', 'cz', 'est', 'hun', 'lat', 'lith', 'pol',
#        'rom', 'slo', 'sle'],
# country_abb: More or less the same as country but in capitals??
# year: 2006 (all!)
# source: 2006_CHES_dataset_means.dta    
# expert: 4-14
# party_id: unique int id
# party: non-unique party abbreviation
# partyname1, partyname2: longer string versions (original/English)
# vote: voteshare in last national election (estimated via seat proprtion where not available for coalition partners)
    # no 0%s here
# vote1: same, but for the overall coalition if party is in one (why not 'coalitional_vote'???)
# family:['radical right', 'cons', 'liberal', 'christdem', 'socialist',
#        'radical left', 'green', 'region/ethnic', 'no family', 'confessional',       'agrarian']
#     FIX - categories need to be renamed!
# govt: categories mislabelled - read ['in government part of year', 0.5, 'in government']
    # SAME PROBLEM, different strings
    # should read ['Party not in government'(/0) ,in government part of year'(/0.5), 'in government'(/1.0)]
    # specific example party 1101 (UK Conservative) is listed as 'in government part of year'
    # entries with .5 check out (verhofstadt's Belgium 1999 6 party coalition!), as does 'in government' entries
    # note - 1999 codebook contains no 'govt' reference!
    # FIX {'in government part of year':0.0, 0.5:0.5, 'in government':1.0}
    
CHES_2006["govt"] = CHES_2006["govt"].replace( {'in government part of year':0.0, 0.5:0.5, 'in government':1.0} )
# govt1: months in government in 2006
CHES_2006["govt1"] = CHES_2006["govt1"]/12
# govt2: no notes in codebook - could it be days? Ranges from 0 -288 - floats - 45 zeroes not 108 (parties never in gov)??
# govt3: similarly confusing!

# intgroup: membership in international political group (2007)
# ['socialist international', 'centrist democratic international',
#        'liberal international', 'global greens',
#        'international democrat union', 'alliance of democrats',
#        'cdi (2) and idu (5)', 'no affiliation']

# epgroup: 
# [        'pes',      'epp-ed',        'alde',  'greens-efa',
#                'uen',     'gue-ngl',          'id',           8.0,       'non-incrits']
# {'pes':"Party of European Socialists (PES)",'epp-ed':"European People’s Party – European Democrats (EPP-ED)",
# 'alde':"Alliance for Liberals and Democrats for Europe (ALDE)",'greens-efa':"European Greens – European Free Alliance (Greens-EFA)",
# 'uen':"Union for Europe of the Nations (UEN)",'gue-ngl':"European United Left – Nordic Green Left (GUE-NGL)",
# 'id':"Independence/Democracy (ID)",'non-incrits':"Non-Incrits (NI)",8.0:"no group"}
            # no group parties, almost all(?) far right
    
    
#### expert responses meaned

# position
# 1-7
# {"strongly in favor":1, "neutral":4, "strongly opposed":7,
#   "strongly opposed to european integration":1,"neutral towards european integration":4,"strongly in favor of european integration":7}
# {"strongly opposed":1,"neutral":4,"strongly in favor":7}
# - std_position

# salience2 (not salience 1 - different range)
# 1-4
# {"one of the most important issues":4,"important issue":3,"minor issue":2} salience1
# {"little importance":2,"some importance":3,"great importance":4} salience2
# -std_salience2



# dissent3 (not dissent1(1-5) or dissent2(1-10))
# 0-10
# {"complete unity":1,"minor dissent":2,'significant dissent':3} dissent1
# no strings in dissent2
# {"party was completely united":0}
# - std_dissent3

# future NOT PRESENT
# 1-5
# {"unification has gone much too far":1, "just the right amount of unification":3}

# benefit
# party leadership’s stance on whether the country has benefited from EU membership (Q4)
# [Note this question corresponds to the standard Eurobarometer “benefit” question.]
# ['benefited','neither benefited nor lost','not benefited']
{'benefited':1,'neither benefited nor lost':2,'not benefited':3, 0.5: 2}
# partyid: 828 - .5 IdV	Di Pietro-Italia dei Valori	Italy of Values - has a value of 0.5, shouldn't be possible
# I'm guessing this should be the intermediate value 2??? BUG!

# eu_ep
# 1-7 (8 is code for 'missing')
# {"strongly opposes expanding ep powers":1,"neutral towards expanding ep powers":4,"strongly favors expanding ep powers":7}
# {"strongly opposed expanding ep powers":1,"neutral towards expanding ep powers":4}
# 7 NULLS???

# eu_fiscal NOT PRESENT
# 1-7 (8 is code for 'missing')
# {"strongly opposes tax harmonization":1,"neutral towards tax harmonization":4,"strongly favors tax harmonization":7}

# eu_employ NOT PRESENT
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common employment policy":1,"neutral towards a common employment policy":4}

# eu_cohesion
# 1-7 (8 is code for 'missing') - codebook has a repeated label typo here!
# {"strongly opposes the eu's cohesion policy":1,"neutral towards the eu's cohesion policy":4,"strongly favors the eu's cohesion policy":7}
# {'neutral':4,strongly favors ':7,}
# 8 NULLS???

# eu_environ NOT PRESENT
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common policy on environment":1,"neutral towards a common policy on environment":4,"strongly favors a common policy on environment":7}

# eu_asylum NOT PRESENT
# 1-7 (8 is code for 'missing')
# expanded list
# {"strongly opposes a common policy on political asylum":1,"neutral towards a common policy on political asylum":4,"strongly favors a common policy on political asylum":7}

# eu_foreign
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common foreign and security policy":1,"neutral towards a common foreign and security policy":4,"strongly favors a common foreign and security policy":7}
# {"strongly opposes":1,neutral":4,}

# lrgen
# 0-10
# {"center":5,"extreme right":10}
# - std_lrgen

# lrecon
# 0-10
# {"center":5}
# - std_lrecon

# galtan
# 0-10
# {"center":5}
# - std_galtan
# 1 NULL???

# eu_intmark
# 1-7 (8 is code for missing)
# {"neutral towards internal market":4}
# {"strongly opposes":1,neutral":4}

# eu_agri NOT PRESENT
# 1-7 (8 is code for missing)
# {"strongly opposes agricultural policy":1,"neutral toward agricultural policy":4,"strongly favors agricultural policy":7}
    
# eu_turkey 8 nulls 1-7
# {"strongly opposes":1,"neutral":4}
        
# pubserv 0-10
# pubserv_salience 0-10
# 2 NULLS???

# dereg
# 3 NULLS
# dereg_sal
# 2 NULLS 

# redist 2 NULLS
{"strongly favors redistribution":0}
# redist_salience 4 NULLS

# civlib 2 NULLS
# civlib_salience 2 NULLS

# lifestyle 3 NULLS
{"strongly opposes liberal policies":10}
# lifestyle_salience 3 NULLS

# religion 3 NULLS
{"strongly supports religious principles in politics":10}
# religion_salience 3 NULLS

# immig 7 NULLS
# immig_salience 4 NULLS

# multicult 10 NULLS
{"strongly favors assimilation":0}
# multicult_salience 5 NULLS

# urban 7 NULLS
# urban_salience 7 NULLS

# cosmo 6 NULLS
{"strongly advocates nationalism":0}
# cosmo_say 6 NULLS

# decentral 4 NULLS
# decentral_salience 10 NULLS

# us 2 NULLS
{"strongly opposes strong us leadership in world affairs":0}
# us_salience 2 NULLS

# minor 3 NULLS 0-10
{"strongly supports more rights for ethnic minorities":0,"strongly opposes more rights for ethnic minorities":10}
# minor_salience 3 NULLS

# all _salience variables 0-10
{"extremely important":10}


describe_messy_column(CHES_2006["dissent3"])

#### LOOOK AT PATTERN OF MISSING VALUES!

0  nulls


party was completely united    1
Name: dissent3, dtype: int64

min:  0.2199999988079071 max:  8.670000076293945


In [780]:
CHES_2010.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2010 = CHES_2010.rename(columns = column_name_replace)
CHES_2010.shape,CHES_2010.columns

['country', 'cname', 'party_id', 'party_name', 'position', 'eu_salience',
       'eu_dissent', 'eu_benefit', 'eu_ep', 'eu_intmark', 'eu_cohesion',
       'eu_foreign', 'eu_turkey', 'lrgen', 'lrecon', 'galtan', 'pubserv',
       'pubserv_salience', 'dereg', 'dereg_salience', 'redist',
       'redist_salience', 'civlib', 'civlib_salience', 'lifestyle',
       'lifestyle_salience', 'religion', 'religion_salience', 'immig',
       'immig_salience', 'multicult', 'multicult_salience', 'urban',
       'urban_salience', 'environment', 'enviro_salience', 'decentral',
       'decentral_salience', 'international_security',
       'international_salience', 'minor', 'minor_salience', 'eu_econ_req',
       'eu_pol_req', 'eu_good_gov', 'source']

# country: suddenly is now code numbers!
# cname: now the lower case short country string previously in country
# party_id: unique int id
CHES_2010["party_id"] = CHES_2010["party_id"].astype(int)
# FIXED: on the basis of changes made in the 1999_2014 trend file!
CHES_2010.loc[CHES_2010["party_id"]==615,"party_id"] = 621
CHES_2010.loc[CHES_2010["party_id"]==841,"party_id"] = 836
CHES_2010.loc[CHES_2010["party_id"]==1019,"party_id"] = 1006
# party_name: short abbreviated form non-unique
# year not present
CHES_2010["year"] = 2010

# position 1-7
# eu_salience 1-4 (same as salience2)
# eu_dissent supposed to be same as dissent3 0-10 ... but we don't see any values above 7 (same questionnaire goes up to 10)
# eu_benefit 1-3 17NULLS
# eu_ep 1-7 17NULLS
# eu_intmark 1-7 17NULLS
# eu_cohesion 1-7 20NULLS
# eu_foreign 1-7 20NULLS
# eu_turkey 1-7 18NULLS

# lrgen 0-10
# lrecon 0-10
# galtan 0-10

# pubserv|dereg|redist|civlib|religion|immig|multicult|urban|environment|decentral|international_security|minor 0-10
# ditto these + _salience 0-10
# only 1NULL in minor_salience

# variable naming inconsistency -> _salience or _sal?
# BUG immasylum_salience instead of multiculturalism_salience!!!

# EU_ECON_REQ, EU_POL_REQ, EU_GOOD_GOV 1-7
# only asked of Croatia and Turkey (17 non-nulls each), and only in 2010


# CHES_2010["position"].value_counts()
describe_messy_column(CHES_2010["eu_good_gov"])
# CHES_2010["eu_benefit"].value_counts()



222  nulls


Series([], Name: eu_good_gov, dtype: int64)

min:  3.0 max:  7.0


In [781]:
CHES_2014.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2014 = CHES_2014.rename(columns = column_name_replace)
CHES_2014.shape,CHES_2014.columns

['country', 'cname', 'party_id', 'party_name', 'eu_position',
        'eu_position_sd', 'eu_salience', 'eu_dissent', 'eu_benefit', 'eu_ep',
        'eu_intmark', 'eu_cohesion', 'eu_foreign', 'eu_turkey', 'eu_budgets',
        'lrgen', 'lrgen_sd', 'lrecon', 'lrecon_salience', 'lrecon_sd', 'galtan',
        'galtan_salience', 'galtan_sd', 'pubserv', 'dereg', 'redist',
        'econ_interven', 'civlib', 'lifestyle', 'religion', 'immig',
        'multicult', 'urban', 'environment', 'decentral',
        'international_security', 'minor', 'cosmo', 'antielite_salience',
        'corrupt_salience', 'mip_one', 'mip_two', 'mip_three', 'source']

# country: suddenly is now code numbers!
# cname: now the lower case short country string previously in country
# party_id: unique int id
CHES_2014["party_id"] = CHES_2014["party_id"].astype(int)


# party_name: short abbreviated form non-unique
# year not present
CHES_2014["year"] = 2014
# eu_position: 1-7 (previous 'position' column names updated for consistency)
# eu_position_sd: 22 NULLS
# eu_salience,eu_dissent: 0-10
# eu_benefit: 1-3
# eu_ep: 1-7 1 NULL
# eu_intmark: 1-7
# eu_cohesion: 1-7 1 NULL
# eu_foreign: 1-7 1 NULL
# eu_turkey: 1-7 2 NULLS
# eu_budgets: 1-7 1 NULL
# lrgen: 0-10
# lrgen_sd: 22 NULLs
# lrecon: 0-10
# lrecon_sd: 23 NULLS
# lrecon_salience: 0-10
# galtan: 0-10
# galtan_sd: 22 NULLS
# galtan_salience: 0-10

# policy position - all 0-10
# 'pubserv', 'dereg', 'redist', 'econ_interven', 'civlib', 'lifestyle', 'religion', 'immig',
# 'multicult', 'urban', 'environment', 'decentral', 'international_security', 'minor', 'cosmo',
# NULLS
# immig                      1
# environment                1
# decentral                  2
# international_security     1

# antielite_salience: 0-10
# corrupt_salience: 0-10

# most important problem structures already solved above
#  'mip_one', 'mip_two', 'mip_three'
# NULLS
# mip_one                   22
# mip_two                   24
# mip_three                 26


CHES_2014["party_id"].value_counts()
describe_messy_column(CHES_2014["mip_one"])

22  nulls


public services vs taxes                                                              66
redistribution                                                                        31
nationalism                                                                           20
environment                                                                           17
anti-elite rhetoric                                                                   16
corruption                                                                            13
immigration                                                                           12
EU integration                                                                        12
ethnic minorities                                                                      9
decentralization                                                                       9
religious principles                                                                   6
deregulation         

min:  nan max:  nan


In [782]:
CHES_1984_1999 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "1984-1999_dataset_means.dta")
CHES_1984_1999["source"] = "1984-1999_dataset_means.dta"

# part of this data comes from here:
# https://www.lsu.edu/faculty/lray2/data/data.html

CHES_1984_1999["lrgen"] = CHES_1984_1999["lrgen"].replace({"center":5}).astype(float)
CHES_1984_1999["lr_gen"] = CHES_1984_1999["lr_gen"].replace({"extreme left":0.0,"extreme right":1.0}).astype(float)
CHES_1984_1999.loc[CHES_1984_1999["lrgen"].isnull(),"lrgen"] = CHES_1984_1999.loc[CHES_1984_1999["lrgen"].isnull(),"lr_gen"]*10

CHES_1984_1999.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_1984_1999 = CHES_1984_1999.rename(columns = column_name_replace)

# dissent2:  1-5 !!!CODEBOOK says Dissent1 and this is obviously the dissent 1 version
CHES_1984_1999 = CHES_1984_1999.rename(columns = {"dissent2":"dissent1"})

CHES_1984_1999.shape,CHES_1984_1999.columns

# CHES_1984_1999.isnull().sum()[CHES_1984_1999.isnull().sum()>0]
# NULLS
# expert         545
# var00001       545
# future         545
# eu_ep          545
# eu_fiscal      545
# eu_employ      545
# eu_cohesion    545
# eu_environ     545
# eu_asylum      545
# eu_foreign     545
# lrgen          545
# lrecon         545
# galtan         545

# the above were all only sampled in year 1999

# lr_gen         132
# no obvious pattern

# pro_anti         2
# position         2
# salience1        2
# dissent2         5


# 'eastwest', 'eumember', 'country', 'expert', 'party_id', 'year',
#         'party', 'vote', 'family', 'position', 'salience1', 'dissent2',
#         'future', 'eu_ep', 'eu_fiscal', 'eu_employ', 'eu_cohesion',
#         'eu_environ', 'eu_asylum', 'eu_foreign', 'lrgen', 'lrecon', 'galtan',
 
# eastwest: "western europe" (all!)
# eumember: 0|1 (mostly 1, presumably == EU member)
# country: ['be', 'dk', 'ge', 'gr', 'esp', 'fr', 'irl', 'it', 'nl', 'uk', 'por', 'aus', 'fin', 'sv']
# expert:  5-15
# party_id: not-unique (covers multiple years)
# year: 1984|1988|1992|1996|1999
# party: abbreviated party initials not unique
# vote: voteshare - contains 101 0.0's ... which doesn't sound right (e.g. maybe code for 'missing value')
# family:['rad right ', 'cons', 'liberal', 'christdem', 'socialist', 'rad left',
#        'green', 'regionalist', 'no family', 'protestant', 'agrarian']
# position: 1-7
{"strongly opposed to ei":1,"neutral, no stance on ei":4,"strongly in favor of ei":7}
# salience1: 1-5
{"no importance, never mentioned":1,
 "minor issue":2,
 "important issue":3,
 "one of the most important issues":4,
 "the most important issue for the party":5,
}
# dissent2:  1-5 !!!CODEBOOK says Dissent1 and this is obviously the dissent 1 version
{"complete unity":1,
"minor dissent":2,
"significant dissent":3,
"evenly split":4,}
# future: 1-5
{"unification has gone much too far":1,
"just the right amount of unification":3}

# eu_ep: 1-7
{"strongly opposed expanding ep powers":1,
"neutral towards expanding ep powers":4,
"strongly favors expanding ep powers":7}

# eu_fiscal: 1-7
{"strongly opposes tax harmonization":1,
"neutral towards tax harmonization":4,
"strongly favors tax harmonization":7}

# eu_employ: 1-7
{"strongly opposes a common employment policy":1,
 "neutral towards a common employment policy":4,}

# eu_cohesion: 1-7
{"strongly opposes tax harmonization":1,
 "neutral towards tax harmonization":4,
 "strongly favors tax harmonization":7}

# eu_environ: 1-7
{"strongly opposes a common policy on environment":1,
"neutral towards a common policy on environment":4,
"strongly favors a common policy on environment":7}

# eu_asylum: 1-7
{"strongly opposes a common policy on political asylum":1}

# eu_foreign: 1-7
{"strongly opposes a common foreign and security policy":1,
 "neutral towards a common foreign and security policy":4,
 "strongly favors a common foreign and security policy":7}
# 'lrgen', 'lrecon', 'galtan',
# lrgen: 0-10
# lrecon: 0-10
# galtan: 0-10

describe_messy_column(CHES_1984_1999["galtan"])
# CHES_1984_1999["family"].value_counts()
# CHES_1984_1999["family"].cat.categories

545  nulls


center    3
Name: galtan, dtype: int64

min:  0.6000000238418579 max:  9.75


In [783]:
CHES_2019 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "CHES2019V3.dta")
CHES_2019["source"] = "CHES2019V3.dta"

CHES_2019.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2019 = CHES_2019.rename(columns = column_name_replace)
CHES_2019.shape,CHES_2019.columns

['country', 'eastwest', 'party', 'party_id', 'eu_position',
        'eu_position_sd', 'eu_salience', 'eu_dissent', 'eu_blur', 'eu_cohesion',
        'eu_foreign', 'eu_intmark', 'eu_budgets', 'eu_asylum', 'lrgen',
        'lrecon', 'lrecon_sd', 'lrecon_salience', 'lrecon_dissent',
        'lrecon_blur', 'galtan', 'galtan_sd', 'galtan_salience',
        'galtan_dissent', 'galtan_blur', 'immig', 'immig_salience',
        'immig_dissent', 'multicult', 'multicult_salience', 'multicult_dissent',
        'redist', 'redist_salience', 'enviro', 'enviro_salience', 'pubserv',
        'dereg', 'econ_interven', 'civlib', 'lifestyle', 'religion', 'minor',
        'cosmo', 'urban', 'protectionism', 'decentral', 'russian_interference',
        'anti_islam_rhetoric', 'people_vs_elite', 'antielite_salience',
        'corrupt_salience', 'members_vs_leadership', 'eu_econ_req',
        'eu_pol_req', 'eu_good_gov', 'source'],

# country: ['be', 'dk', 'ge', 'gr', 'esp', 'fr', 'irl', 'it', 'nl', 'uk', 'por',
#        'aus', 'fin', 'sv', 'bul', 'cz', 'est', 'hun', 'lat', 'lith', 'pol',
#        'rom', 'slo', 'sle', 'cro', 'tur', 'nor', 'swi', 'mal', 'lux', 'cyp', 'ice']
# eastwest: 0.0|1.0
# party: party initials not unique
# party_id: int unique id
CHES_2019["party_id"] = CHES_2019["party_id"].astype(int)
# year not present
CHES_2019["year"] = 2019
# eu_position: 1-7
# - eu_position_sd
# eu_salience: 0-10
# eu_dissent: 0-10 2 NULLS
# eu_blur: 0-10 1 NULL

# eu_cohesion: 1-7 30 NULLS
# eu_foreign: 1-7 31 NULLS
# eu_intmark: 1-7 30 NULLS
# eu_budgets: 1-7 31 NULLS
# eu_asylum: 1-7 31 NULLS

# lrgen: 0-10
# lrecon: 0-10
# - lrecon_sd
# - lrecon_salience: 0-10
# - lrecon_dissent: 0-10 2 NULLs
# galtan: 0-10
# -galtan_sd
# -galtan_salience: 0-10
# -galtan_dissent: 0-10 8 NULLS
# -galtan_blur: 0-10 2 NULLS

# policy positions
# ['immig', 'immig_salience',
#         'immig_dissent', 'multicult', 'multicult_salience', 'multicult_dissent',
#         'redist', 'redist_salience', 'enviro', 'enviro_salience', 'pubserv',
#         'dereg', 'econ_interven', 'civlib', 'lifestyle', 'religion', 'minor',
#         'cosmo', 'urban', 'protectionism', 'decentral', 'russian_interference',
#         'anti_islam_rhetoric'] - all within 0-10, no strings
# NULLS
# immig_dissent          1
# multicult_salience     1
# multicult_dissent      1
# redist                 1
# pubserv                1
# civlib                 1
# minor                  1
# cosmo                  1
# protectionism          1
# decentral              4
# anti_islam_rhetoric    6

# people_vs_elite: 0-10
# antielite_salience: 0-10
# corrupt_salience: 0-10
# members_vs_leadership: 0-10 1NULLS

# eu_econ_req: 0-10 272NULLS
# eu_pol_req: 0-10 272NULLS
# eu_good_gov: 0-10 272NULLS

describe_messy_column(CHES_2019["eu_good_gov"])
# CHES_2019["party_id"].value_counts()
# CHES_2019["country"].cat.categories



KeyError: 'eu_good_gov'

In [784]:
CHES_2007_CAND = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2007_CHES-candidates_dataset_means.dta")
CHES_2007_CAND["source"] = "2007_CHES-candidates_dataset_means.dta"

CHES_2007_CAND.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2007_CAND = CHES_2007_CAND.rename(columns = column_name_replace)
CHES_2007_CAND.shape,CHES_2007_CAND.columns
# 39 x 30

# NULLS - none, which is ... suspect

# ['eumember', 'country', 'country_abb', 'year', 'expert', 'party_id',
#         'party', 'partyname1', 'partyname2', 'membership', 'std_membership',
#         'salience2', 'std_salience2', 'dissent3', 'std_dissent3',
#         'require_econ', 'require_pol', 'require_gov', 'lrgen', 'std_lrgen',
#         'lrecon', 'std_lrecon', 'galtan', 'std_galtan', 'pubserv', 'redist',
#         'religion', 'cosmo', 'minor', 'source']

# eumember: 'other' (all!)
# country: codes [30,31,32,33,34]
# country_abb: [CRO,SER,MKD,BID,TUR] - like cname, but allcaps
# year: 2007 (all!)
# expert: 5-9
# party_id: unique int id
# party: party initials not unique
# partyname1: original language full name unique(in this small dataset)
# partyname2: english version - not unique

# membership: 1-7
{"neutral":4}
# - std_membership
# salience2: 1-4
{"little importance":2,
 "some importance":3,
 "great importance":4,}
# - std_salience2
# dissent3: 0-10
# - std_dissent3

# These match the eu_req_econ/pol/good_gov variables but the scale needs to be transformed from 1-7 -> 0-10
# require_econ: 1-7
# require_pol: 1-7
{"neutral":4}
# require_gov: 1-7
{"neutral":4}

# lrgen: 0-10
{"center":5}
# -std_lrgen
# lrecon: 0-10
{"center":5}
# -std_lrecon
# galtan: 0-10
# -std_galtan

# policies
# pubserv: 0-10
# redist: 0-10
# religion: 0-10
# cosmo: 0-10
# minor: 0-10
{"strongly supports more rights for ethnic minorities":0}




describe_messy_column(CHES_2007_CAND["pubserv"])
# CHES_2007_CAND["membership"].value_counts()
# CHES_2007_CAND["country"].cat.categories

CHES_2007_CAND.shape,CHES_2007_CAND.columns

0  nulls


Series([], Name: pubserv, dtype: int64)

min:  1.0 max:  8.25


((39, 30),
 Index(['eumember', 'country', 'country_abb', 'year', 'expert', 'party_id',
        'party', 'partyname1', 'partyname2', 'membership', 'membership_sd',
        'salience2', 'salience2_sd', 'dissent3', 'dissent3_sd', 'eu_econ_req',
        'eu_pol_req', 'eu_good_gov', 'lrgen', 'lrgen_sd', 'lrecon', 'lrecon_sd',
        'galtan', 'galtan_sd', 'pubserv', 'redist', 'religion', 'cosmo',
        'minor', 'source'],
       dtype='object'))

In [785]:
CHES_2014_UKR = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2014_Candidates_Ukraine_Means.dta")
CHES_2014_UKR["source"] = "2014_Candidates_Ukraine_Means.dta"

CHES_2014_UKR.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2014_UKR = CHES_2014_UKR.rename(columns = column_name_replace)
CHES_2014_UKR.shape,CHES_2014_UKR.columns
# 62 x 38
# NULLS
# Ukraine specific questions
# conflict_res     50
# region_status    50
# languages        50
# russia_role      50
# oligarchs        50





# 'country', 'cname', 'party_id', 'party_name', 'membership',
#        'eu_salience', 'eu_dissent', 'require_econ', 'require_pol',
#        'require_gov', 'lrgen', 'lrecon', 'lrecon_salience', 'galtan',
#        'galtan_salience', 'pubserv', 'dereg', 'redist', 'econ_interven',
#        'civlib', 'lifestyle', 'religion', 'immig', 'multicult', 'urban',
#        'enviro', 'decentral', 'international_security', 'minor', 'cosmo',
#        'antielite_salience', 'corrupt_salience', 'conflict_res',
#        'region_status', 'languages', 'russia_role', 'oligarchs', 'source']

# country: codes[30,32,33,41,42,43,44]
# cname: [bih,ukr,ser,mne,kos,alb,mkd]
# no year column - but implicitly 2014!
CHES_2014_UKR["year"] = 2014
# no expert column
# party_id: unique int id
CHES_2014_UKR["party_id"] = CHES_2014_UKR["party_id"].astype(int)
# party_name: party initials not unique

# membership: 1-7
# eu_salience: 0-10
# CODEBOOK BUG erroneously says there should be an eu_salience_sd
# eu_dissent: 0-10
# require_econ,require_pol,require_gov: 1-7

# lrgen,lrecon,galtan:0-10
# lrecon_salience,galtan_salience: 0-10
# CODEBOOK BUG incorrectly says there should be lrgen_sd,lrecon_sd,galtan_sd

# policies - all fine 0-10
# 'pubserv', 'dereg', 'redist', 'econ_interven',
#        'civlib', 'lifestyle', 'religion', 'immig', 'multicult', 'urban',
#        'enviro', 'decentral', 'international_security', 'minor', 'cosmo',

# salience - both fine 0-10
#        'antielite_salience', 'corrupt_salience',

# Ukraine specific: all fine 0-10
# 'conflict_res', 'region_status', 'languages', 'russia_role', 'oligarchs'

describe_messy_column(CHES_2014_UKR["oligarchs"])
# CHES_2014_UKR["membership"].value_counts()
# CHES_2014_UKR["country"].cat.categories


# CHES_2014_UKR.isnull().sum()[CHES_2014_UKR.isnull().sum()>0]
# CHES_2014_UKR.columns


50  nulls


Series([], Name: oligarchs, dtype: int64)

min:  4.0625 max:  9.333333


In [786]:
CHES_2017 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "CHES_means_2017.dta")
CHES_2017["source"] = "CHES_means_2017.dta"

# electionyear[new],seat[new],
# bugged columns 'multiculture', 'multiculture_salience', 'multiculture_dissent' - empty
# ditto ['eu_position','eu_budget','immigrate_salience']
# party_name column also bugged - only ""
CHES_2017 = CHES_2017.drop(["party_name",'eu_position','eu_budget','immigrate_salience'],axis=1)

# experts -> expert
# _clear -> _blur? - actually very good case for this 60-70% negative correlation between X_clear and X_blur
# it is only 6 variables: lrecon_clear	galtan_clear	galtan_new_clear	eu_blur	lrecon_blur	galtan_blur
# only 2 overlapping pairs

CHES_2017.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2017 = CHES_2017.rename(columns = column_name_replace)

CHES_2017.shape,CHES_2017.columns

# NULLS
# vote                  6
# seat                  5
# eu_asylum             1
# eu_budgets            1
# galtan_clear          1
# immig                 1
# immig_dissent         1
# multicult             1
# multicult_salience    1
# multicult_dissent     1
# galtan_new_clear      1

# country: ['ger', 'gr', 'esp', 'fr', 'it', 'nl', 'uk', 'por', 'swe', 'cz', 'est', 'hun', 'pol', 'slo']
# year: 2017 (all!)
# expert: 12-22
CHES_2017["expert"] = CHES_2017["expert"].astype(int)
# party_id: unique int id
CHES_2017["party_id"] = CHES_2017["party_id"].astype(int)
# party: initials not unique
# vote: voteshare (0.01-42.4) voteshare in most recent prior election
# seat: seats (0-58.8) seats in most recent prior election (.8 -> coalition fraction???)
# electionyear: 2013|2014|2015|2016|2017
CHES_2017["electionyear"] = CHES_2017["electionyear"].astype(int)
# family: party family
# govt: 0/0.5/1.0
# eu_position: 1-7
# eu_salience:0-10
# eu_dissent:0-10

# eu_asylum:1-7 1NULL
# eu_budgets: 1-7 1NULL
# lrgen,lrecon,galtan: 0-10 
# lrecon_clear, galtan_clear(1NULL): 0-10
# lrecon_salience, galtan_salience: 0-10
# galtan_new, galtan_new_clear(1NULL), galtan_new_salience: 0-10

# policies: all fine 0-10
# 'immig', 'immig_salience', 'immig_dissent',
# 'multicult', 'multicult_salience', 'multicult_dissent', 'minor',
# 'people_vs_elite', 'antielite_salience', 'corrupt_salience', 'members_vs_leadership'


describe_messy_column(CHES_2017["galtan_new_salience"])
# CHES_2017["govt"].value_counts()
# CHES_2017["country"].cat.categories
# CHES_2017.shape,CHES_2017.columns




0  nulls


Series([], Name: galtan_new_salience, dtype: int64)

min:  3.0 max:  9.733334


In [16]:
CHES_1999_2014[CHES_1999_2014["year"]==1999].shape,\
    CHES_1999_2014[CHES_1999_2014["year"]==2002].shape,\
    CHES_1999_2014[CHES_1999_2014["year"]==2006].shape,\
    CHES_1999_2014[CHES_1999_2014["year"]==2010].shape,\
    CHES_1999_2014[CHES_1999_2014["year"]==2014].shape,

((142, 73), (171, 73), (188, 73), (203, 73), (245, 73))

In [787]:
df = pd.concat([CHES_1984_1999,CHES_1999,CHES_2002,CHES_2006,CHES_2010,CHES_2014,CHES_2007_CAND,CHES_2014_UKR,CHES_2017,CHES_2019],axis=0)
cols = df.columns

In [788]:
# df[["country","country_abb","cname","source"]]

# df["source"][df["country"].apply(could_be_int)].value_counts()
# df[["country","cname","country_abb"]][df["country"].apply(could_be_int)].drop_duplicates()["country"].value_counts()

# # 24.0        2
# # 10.0        2
# # 1.0         2

# country_triples = df[["country","cname","country_abb"]]\
#     .replace({"neth":"net","bel":"be"})\
#     .applymap(lambda x: x.lower() if not isinstance(x,float) else x)\
#     .drop_duplicates()
# #[df["country"]\
# #     .drop_duplicates()
# #     .apply(lambda x: x in [1,10,24])]\
    




In [789]:
ches_country_code_dict = {1: 'be', 2: 'den', 3: 'ger', 4: 'gre', 5: 'spa', 6: 'fra', 7: 'ire', 8: 'it', 10: 'net', 11: 'uk',
 12: 'por', 13: 'aus', 14: 'fin', 16: 'swe', 20: 'bul', 21: 'cze', 22: 'est', 23: 'hun', 24: 'lat', 25: 'lith', 26: 'pol',
 27: 'rom', 28: 'slo', 29: 'sle', 30: 'bih', 31: 'cro', 32: 'mkd', 33: 'ser', 34: 'tur', 35: 'nor', 36: 'swi', 37: 'mal',
 38: 'lux', 40: 'cyp', 41: 'alb', 42: 'kos', 43: 'mne', 44: 'ukr'}

trans_to_party_facts_countries = {"den":"dnk","ger":"deu","gre":"grc","spa":"esp","ire":"irl","it":"ita",
"net":"nld","uk":"gbr","por":"prt","aus":"aut","bul":"bgr","lat":"lva",
"lith":"ltu","pol":"prt","rom":"rou","slo":"svk","sle":"svn","cro":"hrv",
"ser":"srb","swi":"che","mal":"mlt","kos":"xkx",
"be":"bel","be-wall":"bel","be-fl":"bel",
"cz":"cze","czech":"cze",
"hung":"hun","slovak":"svk","sloven":"svn","neth":"nld","nl":"nld",
"ice":"isl","fr":"fra","ge":"deu","gr":"grc","dk":"dnk","sv":"swe",}

def country_harmonize(x):
    if could_be_int(x):
        x = int(x)
    if isinstance(x, int):
        x = ches_country_code_dict[x]
    if x in trans_to_party_facts_countries.keys():
        x = trans_to_party_facts_countries[x]
    return x

In [790]:
partyfacts_countries = pd.read_csv(dataset_folder+"PartyFacts"+os.sep+ "partyfacts-countries.csv")
partyfacts_countries["country"] = partyfacts_countries["country"].apply(lambda x: x.lower())
partyfacts_countries = partyfacts_countries.set_index("country")

df["country"] = df["country"].apply(country_harmonize)
df["country_name"] = df["country"].apply(lambda x: partyfacts_countries.loc[x,"name"])
df["country_continent"] = df["country"].apply(lambda x: partyfacts_countries.loc[x,"continent"])
df["country_region"] = df["country"].apply(lambda x: partyfacts_countries.loc[x,"region"])

def lower_if_not_float(x):
    return x.lower() if not isinstance(x,float) else x

df["country_sub_region"] = np.nan
df.loc[df["country_abb"].apply(lambda x: lower_if_not_float(x))=="be-wall","country_sub_region"] = "Wallonia"
df.loc[df["country_abb"].apply(lambda x: lower_if_not_float(x))=="be-fl","country_sub_region"] = "Flanders"

# drop these! "country_abb","cname"
df = df.drop(["country_abb","cname"],axis=1)

In [ ]:
# 2014_CHES_dataset_means-2.dta             268
# 2010_CHES_dataset_means.dta               239
# 2014_Candidates_Ukraine_Means.dta          62
# 2007_CHES-candidates_dataset_means.dta     39


In [599]:
## quickly grab the expert-level datasets
filename = "1999_CHES_dataset_expert-level.dta"
CHES_1999_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_1999_explvl["source"] = filename

filename = "2002_CHES_dataset_expert-level_east.dta"
CHES_2002_explvl_east = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2002_explvl_east["source"] = filename

filename = "2002_CHES_dataset_expert-level_west.dta"
CHES_2002_explvl_west = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2002_explvl_west["source"] = filename

filename = "2006_CHES_dataset_expert-level.dta"
CHES_2006_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2006_explvl["source"] = filename

filename = "2010_CHES_dataset_expert-level.dta"
CHES_2010_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2010_explvl["source"] = filename

filename = "2014_CHES_dataset_expert-level.dta"
CHES_2014_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2014_explvl["source"] = filename

filename = "combined_experts_2017.dta"
CHES_2017_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2017_explvl["source"] = filename

filename = "2014_Candidates_Ukraine_Experts.dta"
CHES_2014_UKR_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2014_UKR_explvl["source"] = filename

filename = "2007_CHES-candidates_dataset_expert-level.dta"
CHES_2007_CAND_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2007_CAND_explvl["source"] = filename

# 1984-1999 and 2019 expert level datasets missing

In [607]:
for ds in [CHES_1999_explvl,CHES_2002_explvl_east,CHES_2002_explvl_west,
           CHES_2006_explvl,CHES_2010_explvl,CHES_2014_explvl,
           CHES_2017_explvl,CHES_2014_UKR_explvl,CHES_2007_CAND_explvl]:
    print(ds["source"].unique()[0],"\n",ds.columns)


1999_CHES_dataset_expert-level.dta 
 Index(['expert', 'country', 'partyname', 'party_', 'q1', 'q2', 'q3', 'q4',
       'q5', 'q6a', 'q6b', 'q6c', 'q6d', 'q6e', 'q6f', 'q6g', 'q8a', 'q8b',
       'q8c', 'source'],
      dtype='object')
2002_CHES_dataset_expert-level_east.dta 
 Index(['expert', 'country', 'party_na', 'party__', 'q1', 'q2', 'q3', 'q4',
       'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13a', 'q13b',
       'q13c', 'q13d', 'q13e', 'q13f', 'q13g', 'q13h', 'q14', 'q15', 'q16',
       'expert1', 'country1', 'party_n1', 'party__1', 'source'],
      dtype='object')
2002_CHES_dataset_expert-level_west.dta 
 Index(['expert', 'country', 'party_na', 'party__', 'q1', 'q2', 'q3', 'q4',
       'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q12_5pt_', 'q13a',
       'q13b', 'q13c', 'q13d', 'q13e', 'q13f', 'q13g', 'q13h', 'q14', 'q15',
       'q16', 'source'],
      dtype='object')
2006_CHES_dataset_expert-level.dta 
 Index(['country_id', 'party_id', 'party', 'expert_id', 

In [820]:
df = pd.concat([CHES_1999,CHES_2002,CHES_2006,CHES_2010,CHES_2014,CHES_2007_CAND,CHES_2014_UKR,CHES_2017,CHES_2019],axis=0)
df.isnull().sum().head(50)

eastwest                770
eumember                978
country                   0
year                      0
expert                  846
party_id                  0
party                   569
vote                    896
family                  885
govt                    885
eu_position             101
salience1              1376
dissent1               1376
future                 1376
eu_ep                   535
eu_fiscal              1376
eu_employ              1205
eu_cohesion             292
eu_environ             1205
eu_asylum               828
eu_foreign              285
lrgen                     0
lrecon                    0
galtan                    1
source                    0
partyname1             1120
partyname2             1120
vote1                  1164
position_sd            1159
salience2              1120
eu_intmark              422
eu_agri                1347
enlargw                1420
enlarge                1445
dissent2               1347
eu_ep_dissent       

In [821]:
salience1_dict = {"no importance, never mentioned":1,
 "minor issue":2,
 "important issue":3,
 "one of the most important issues":4,
 "the most important issue for the party":5,
}

df["salience1"] = ( df["salience1"].replace(salience1_dict).astype(float) - 1)/4

salience2_dict = {"little importance":2,
 "some importance":3,
 "great importance":4,}

df["salience2"] = (df["salience2"].replace(salience2_dict).astype(float)-1)/3

df["eu_salience_sd"] = df["salience2_sd"]/3

df["eu_salience"] = df["eu_salience"].astype(float)/10.0

df.loc[df["salience1"].notnull(),"eu_salience"] = df.loc[df["salience1"].notnull(),"salience1"]
df.loc[df["salience2"].notnull(),"eu_salience"] = df.loc[df["salience2"].notnull(),"salience2"]

df = df.drop(["salience1","salience2","salience2_sd"],axis=1,)

In [822]:

# dissent1, dissent2, dissent3, dissent3_sd, eu_dissent

# 1-5
dissent1_dict = {"complete unity":1,"minor dissent":2,'significant dissent':3}

df["dissent1"] = ( df["dissent1"].replace(dissent1_dict).astype(float) - 1)/4

# dissent2 1-10
df["dissent2"] = (df["dissent2"]-1)/9

# dissent3 0-10
dissent3_dict = {"party was completely united":0}
df["dissent3"] = df["dissent3"].replace(dissent3_dict).astype(float)/10.0
df["eu_dissent_sd"] = df["dissent3_sd"]/10.0
# eu_dissent 0-10
df["eu_dissent"] = df["eu_dissent"]/10.0

df.loc[df["dissent1"].notnull(),"eu_dissent"] = df.loc[df["dissent1"].notnull(),"dissent1"]
df.loc[df["dissent2"].notnull(),"eu_dissent"] = df.loc[df["dissent2"].notnull(),"dissent2"]
df.loc[df["dissent3"].notnull(),"eu_dissent"] = df.loc[df["dissent3"].notnull(),"dissent3"]

df = df.drop(["dissent1","dissent2","dissent3","dissent3_sd"],axis=1,)

In [823]:
# # 2019 ['eu_econ_require', 'eu_political_require', 'eu_googov_require']
# 'eu_econ_req', 'eu_pol_req','eu_good_gov': 1-7 CHES_2007_CAND 1-7,CHES_2014_UKR -17,CHES_2010)

# df["eu_econ_req"].min(),df["eu_econ_req"].max()
# df["require_econ"].min(),df["require_econ"].max()

# looks like a CODEBOOK bug - CHES_2019 also seems to be 1-7 not 0-10

In [824]:
req_dict = {"neutral":4}

df["eu_econ_req"] = (df["eu_econ_req"].replace(req_dict).astype(float)-1)/6.0
df["eu_pol_req"] = (df["eu_pol_req"].replace(req_dict).astype(float)-1)/6.0
df["eu_good_gov"] = (df["eu_good_gov"].replace(req_dict).astype(float)-1)/6.0

df["eu_econ_require"] = df["eu_econ_require"]/10.0
df["eu_political_require"] = df["eu_political_require"]/10.0
df["eu_googov_require"] = df["eu_googov_require"]/10.0

# print(df["eu_econ_req"].min(),df["eu_econ_req"].max(),df["eu_pol_req"].min(),df["eu_pol_req"].max(),df["eu_good_gov"].min(),df["eu_good_gov"].max())
# print(df["eu_econ_require"].min(),df["eu_econ_require"].max(),df["eu_political_require"].min(),df["eu_political_require"].max(),df["eu_googov_require"].min(),df["eu_googov_require"].max())

df.loc[df["eu_econ_require"].notnull(),"eu_econ_req"] = df.loc[df["eu_econ_require"].notnull(),"eu_econ_require"]
df.loc[df["eu_political_require"].notnull(),"eu_pol_req"] = df.loc[df["eu_political_require"].notnull(),"eu_political_require"]
df.loc[df["eu_googov_require"].notnull(),"eu_good_gov"] = df.loc[df["eu_googov_require"].notnull(),"eu_googov_require"]

df = df.drop(["eu_econ_require","eu_political_require","eu_googov_require"],axis=1,)

In [827]:
list(df.columns)

['eastwest',
 'eumember',
 'country',
 'year',
 'expert',
 'party_id',
 'party',
 'vote',
 'family',
 'govt',
 'eu_position',
 'future',
 'eu_ep',
 'eu_fiscal',
 'eu_employ',
 'eu_cohesion',
 'eu_environ',
 'eu_asylum',
 'eu_foreign',
 'lrgen',
 'lrecon',
 'galtan',
 'source',
 'partyname1',
 'partyname2',
 'vote1',
 'position_sd',
 'eu_intmark',
 'eu_agri',
 'enlargw',
 'enlarge',
 'eu_ep_dissent',
 'eu_intmark_dissent',
 'eu_employ_dissent',
 'eu_agri_dissent',
 'eu_cohesion_dissent',
 'eu_environ_dissent',
 'eu_asylum_dissent',
 'enlargw_dissent',
 'enlarge_dissent',
 'lrgen_sd',
 'lrecon_sd',
 'galtan_sd',
 'country_abb',
 'intgroup',
 'epgroup',
 'govt1',
 'govt2',
 'govt3',
 'benefit',
 'eu_turkey',
 'pubserv',
 'pubserv_salience',
 'dereg',
 'dereg_salience',
 'redist',
 'redist_salience',
 'civlib',
 'civlib_salience',
 'lifestyle',
 'lifestyle_salience',
 'religion',
 'religion_salience',
 'immig',
 'immig_salience',
 'multicult',
 'multicult_salience',
 'urban',
 'urban_salie

In [837]:
df["source"][df["year"].isnull()].value_counts()

df["uid"] = (df["year"].astype(int).astype(str)+"_"+df["party_id"].astype(int).astype(str))
df["uid"].value_counts()

2010_3503    1
2019_1014    1
2006_504     1
2010_701     1
2010_2411    1
            ..
2019_2906    1
2006_802     1
2019_4501    1
2002_2907    1
2014_1016    1
Name: uid, Length: 1518, dtype: int64

In [676]:
CHES_1999_2014 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "1999-2014_CHES_dataset_means-3.dta")
CHES_1999_2014["source"] = "1999-2014_CHES_dataset_means-3.dta"

# describe_messy_column(CHES_1999_2014["galtan_new_salience"])
# CHES_1999_2014["govt"].value_counts()
# CHES_1999_2014["country"].cat.categories
CHES_1999_2014.shape,CHES_1999_2014.columns

((949, 73),
 Index(['country', 'eastwest', 'eumember', 'year', 'expert', 'party_id',
        'cmp_id', 'party', 'vote', 'seat', 'electionyear', 'epvote', 'family',
        'govt', 'position', 'eu_salience', 'eu_dissent', 'eu_benefit', 'eu_ep',
        'eu_fiscal', 'eu_intmark', 'eu_employ', 'eu_budgets', 'eu_agri',
        'eu_cohesion', 'eu_environ', 'eu_asylum', 'eu_foreign', 'eu_turkey',
        'lrgen', 'lrecon', 'galtan', 'lrecon_salience', 'galtan_salience',
        'spendvtax', 'spendvtax_salience', 'deregulation', 'dereg_salience',
        'redistribution', 'redist_salience', 'civlib_laworder',
        'civlib_salience', 'sociallifestyle', 'social_salience',
        'religious_principle', 'relig_salience', 'immigrate_policy',
        'immigra_salience', 'multiculturalism', 'multicult_salience',
        'urban_rural', 'urban_salience', 'environment', 'enviro_salience',
        'cosmo', 'cosmo_salience', 'regions', 'region_salience',
        'international_security', 'internation

In [677]:
CHES_1999_2014.isnull().sum().head(50)

CHES_1999_2014["uid"] = (CHES_1999_2014["year"].astype(int).astype(str)+"_"+CHES_1999_2014["party_id"].astype(int).astype(str))

In [693]:


salience1_dict = {"no importance, never mentioned":1,
 "minor issue":2,
 "important issue":3,
 "one of the most important issues":4,
 "the most important issue for the party":5,
}

temp = df["salience1"].replace(salience1_dict).astype(float)
temp = (temp-1)/4
df["salience1"] = temp




(0.125, 0.75)

(0.2433333396911621, 1.0)

In [708]:
# df[["salience1","salience2","eu_salience"]].notnull().sum(axis=1).value_counts()
# no overlap/gaps!

1    1008
dtype: int64

In [713]:
df[[x for x in df.columns if "salience2" in x]]




,salience2,salience2_sd
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
263,NaN,NaN
264,NaN,NaN
265,NaN,NaN
266,NaN,NaN


In [828]:
shared_cols = set.intersection(set(CHES_1999_2014.columns),set(df.columns))
temp = df[shared_cols].isnull().sum() - CHES_1999_2014[shared_cols].isnull().sum()
temp[temp>59]

# year, source, party_id
# govt, eumember, party, eastwest, expert, vote

# eu_salience, eu_dissent -> being fixed separately, hopefully!

# cmp_id,seat,electionyear,epvote,


corrupt_salience            75
eu_environ                 568
international_salience     533
econ_interven              207
us_salience                569
mip_three                  554
govt                       885
lrecon_salience             75
eumember                   978
civlib_salience            533
eu_fiscal                  569
redist_salience            256
dereg_salience             533
party                      569
family                     885
us                         569
eu_cohesion                284
international_security     449
mip_two                    554
eu_agri                    569
eu_asylum                  192
eu_foreign                 285
enviro_salience            256
electionyear              1386
eu_employ                  565
vote                       846
eastwest                   770
mip_one                    554
cosmo_salience             569
eu_turkey                  528
urban_salience             533
eu_benefit                 527
galtan_s

In [833]:
CHES_1999_2014[["uid", "govt", "eumember", "party", "eastwest", "expert", "vote",
                        "cmp_id","seat","electionyear","epvote"]]#.set_index("uid")

,govt,eumember,party,eastwest,expert,vote,cmp_id,seat,electionyear,epvote
uid,,,,,,,,,,
1999_102,in government,EU member state,PS,west,9.0,10.2,21322.0,12.7,1999.0,9.59
1999_103,in government,EU member state,SP,west,9.0,9.6,21321.0,9.3,1999.0,8.84
1999_104,0.5,EU member state,ECOLO,west,9.0,7.3,21111.0,7.3,1999.0,8.44
1999_105,0.5,EU member state,AGALEV,west,9.0,7.0,21112.0,6.0,1999.0,7.46
1999_106,0.5,EU member state,PRL,west,9.0,7.7,21422.0,9.0,1999.0,6.69
...,...,...,...,...,...,...,...,...,...,...
2014_4002,in government,EU member state,EVROKO,east,4.0,3.9,55430.0,3.6,2011.0,0.00
2014_4003,not in government,EU member state,AKEL,east,4.0,32.7,55321.0,33.9,2011.0,27.00
2014_4004,0.5,EU member state,DIKO,east,4.0,15.8,55422.0,16.1,2011.0,10.80


In [847]:
cols = ["uid", "govt", "eumember", "party", "eastwest", "expert", "vote",
                        "cmp_id","seat","electionyear","epvote"]
temp = CHES_1999_2014[cols]
temp.columns = "trend_"+temp.columns

df = df.merge(right=temp, how='left', left_on="uid", right_on="trend_uid",)

In [851]:
df[["govt","trend_govt"]].drop_duplicates()

govt_dict = {"in government":1.0,"not in government":0.0}
govt_null = df["govt"].isnull()
df.loc[govt_null,"govt"] = df.loc[govt_null,"trend_govt"].replace(govt_dict)


0.0    622
1.0    250
0.5     77
Name: trend_govt, dtype: int64

In [869]:
eastwest_dict = {"east":0.0,"west":1.0}
eastwest_null = df["eastwest"].isnull()
df.loc[eastwest_null,"eastwest"] = df.loc[eastwest_null,"trend_eastwest"].replace(eastwest_dict)

df["eastwest"] = df["eastwest"].replace(eumember_dict)

df["eastwest"].value_counts()

Yes     412
west    355
No      283
east    146
Name: eastwest, dtype: int64

In [872]:
# no disagreement
# df[["expert","trend_expert"]].drop_duplicates()
expert_null = df["expert"].isnull()
df.loc[expert_null,"expert"] = df.loc[expert_null,"trend_expert"]

In [886]:
# some disagreement

# df[["vote","trend_vote"]].drop_duplicates()
# temp = (df["vote"]-df["trend_vote"])
# temp[temp.abs()>5]

# df.loc[[288,442]][["year","country","party_id","vote","trend_vote","party","trend_party","electionyear"]]

# after some basic testing, looks like vote in the trend file is better

vote_non_null = df["trend_vote"].notnull()
df.loc[vote_non_null,"vote"] = df.loc[vote_non_null,"trend_vote"]

df["cmp_id"] = df["trend_cmp_id"]
df["epvote"] = df["trend_epvote"]

,year,country,party_id,vote,trend_vote,party,trend_party,electionyear
288,2002.0,rom,2701,43.099998,36.6,PSD,PSD,NaN
442,2006.0,est,2201,31.930000,7.3,IRL,IRL,NaN


In [890]:
# seat  - no overlap

seat_null = df["seat"].isnull()
df.loc[seat_null,"seat"] = df.loc[seat_null,"trend_seat"]


Series([], dtype: float64)

In [892]:
df[["electionyear","trend_electionyear"]].drop_duplicates()
# electionary  - no overlap

electionyear_null = df["electionyear"].isnull()
df.loc[electionyear_null,"electionyear"] = df.loc[electionyear_null,"trend_electionyear"]


,electionyear,trend_electionyear
0,NaN,1999.0
14,NaN,1998.0
34,NaN,1996.0
55,NaN,1997.0
152,NaN,2001.0
158,NaN,2002.0
164,NaN,2000.0
313,NaN,2003.0
323,NaN,2005.0
338,NaN,2004.0


In [864]:
df[["party","trend_party"]].isnull().sum(axis=1).value_counts()
# leave as is!

1    896
0    501
2    121
dtype: int64

In [857]:
eumember_dict = {"eu member state":"Yes","EU member state":"Yes",1:"Yes",0:"No","other":"Other"}
eumember_null = df["eumember"].isnull()

df.loc[eumember_null,"eumember"] = df.loc[eumember_null,"trend_eumember"]
df["eumember"] = df["eumember"].replace(eumember_dict)

df["eumember"].value_counts()

Yes      862
No        73
Other     53
Name: eumember, dtype: int64

In [845]:
df.isnull().sum().sort_values().head(50)


# CHES_1999_2014[[x for x in CHES_1999_2014.columns if x not in list(shared_cols)]].isnull().sum()

# 

country                 0
year                    0
eu_salience             0
party_id                0
source                  0
lrecon                  0
lrgen                   0
uid                     0
galtan                  1
eu_dissent              2
eu_position           101
eu_foreign            285
eu_cohesion           292
minor                 317
immig                 361
multicult             363
eu_intmark            422
pubserv               446
religion              448
redist                448
civlib                487
lifestyle             487
dereg                 487
urban                 491
decentral             494
eu_ep                 535
trend_eumember        569
trend_electionyear    569
party                 569
trend_expert          569
trend_eastwest        569
trend_party           569
trend_govt            569
trend_uid             569
galtan_sd             598
lrecon_sd             598
trend_vote            619
trend_seat            623
enviro      

In [846]:
df.merge(right=temp, how='left', left_on="uid", right_on="trend_uid",).isnull().sum().sort_values().tail(50)

lrecon_blur               1244
anti_islam_rhetoric       1247
galtan_dissent            1249
mip_one                   1272
mip_two                   1274
mip_three                 1276
international_salience    1279
country_abb               1291
eu_salience_sd            1291
eu_dissent_sd             1291
benefit                   1330
govt1                     1330
epgroup                   1331
intgroup                  1331
us_salience               1332
us                        1332
cosmo_salience            1335
eu_cohesion_dissent       1347
eu_ep_dissent             1347
eu_intmark_dissent        1347
eu_employ_dissent         1347
eu_agri_dissent           1347
eu_environ_dissent        1347
eu_agri                   1347
eu_asylum_dissent         1347
eu_fiscal                 1376
future                    1376
galtan_new_salience       1386
lrecon_clear              1386
electionyear              1386
galtan_new                1386
galtan_new_clear          1387
galtan_c

In [841]:
"trend"+temp.columns

Index(['trendtrenduid', 'trendtrendgovt', 'trendtrendeumember',
       'trendtrendparty', 'trendtrendeastwest', 'trendtrendexpert',
       'trendtrendvote', 'trendtrendcmp_id', 'trendtrendseat',
       'trendtrendelectionyear', 'trendtrendepvote'],
      dtype='object')

In [740]:
CHES_1999_2014[ CHES_1999_2014["uid"].apply(lambda x : x not in df["uid"].values) ]

# 710 in CHES_1999 -> 708 in CHES1999_2014

# difference between DS and SP is genuinely confusing!

# 621, 31630, fr, NC (party_id CHES_2010 = 615)
# 836, NaN, it, PdCl (party_id CHES_2010 = 841)
# 1006, 22952, nl, SGP (party_id CHES_2010 = 1019)

,country,eastwest,eumember,year,expert,party_id,cmp_id,party,vote,seat,...,econ_interven,nationalism,antielite_salience,corrupt_salience,mip_one,mip_two,mip_three,chesversion,source,uid


In [656]:
df[ df["uid"].apply(lambda x : x not in CHES_1999_2014["uid"].values) ]

,eastwest,eumember,country,year,expert,party_id,party,vote,family,govt,...,lrecon_sd,galtan_salience,galtan_sd,econ_interven,antielite_salience,corrupt_salience,mip_one,mip_two,mip_three,uid
203,NaN,NaN,31,2010,NaN,3101,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010_3101
204,NaN,NaN,31,2010,NaN,3102,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010_3102
205,NaN,NaN,31,2010,NaN,3103,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010_3103
206,NaN,NaN,31,2010,NaN,3104,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010_3104
207,NaN,NaN,31,2010,NaN,3105,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010_3105
208,NaN,NaN,31,2010,NaN,3106,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010_3106
209,NaN,NaN,31,2010,NaN,3107,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010_3107
210,NaN,NaN,31,2010,NaN,3108,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010_3108
211,NaN,NaN,31,2010,NaN,3109,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010_3109
212,NaN,NaN,31,2010,NaN,3110,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010_3110
